In [18]:
import matplotlib.pyplot as plt
import pymc3 as pm

In [20]:
import pyjags

ModuleNotFoundError: No module named 'jags'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import pymc3 as pm
import theano.tensor as tt
import scipy
from scipy import optimize

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [14]:
x1 = np.repeat(1, 12)
x2 = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
x3 = np.array([23 ,22 ,22 ,25 ,27 ,20 ,31 ,23 ,27 ,28 ,22 ,24])
x4 = x2 * x3
y = np.array([-0.87, -10.74, -3.27, -1.97, 7.50, -7.25, 17.05, 4.96, 10.40, 11.05, 0.26, 2.51])
X = np.column_stack([x1, x2, x3, x4])
n = X.shape[0]
p = X.shape[1]

olsresult = sm.OLS(y, X[:,1:]).fit()
beta_ols = olsresult.params
sigma2head_ols= sum(olsresult.resid ** 2) / (n-p)


[[ 1  0 23  0]
 [ 1  0 22  0]
 [ 1  0 22  0]
 [ 1  0 25  0]
 [ 1  0 27  0]
 [ 1  0 20  0]
 [ 1  1 31 31]
 [ 1  1 23 23]
 [ 1  1 27 27]
 [ 1  1 28 28]
 [ 1  1 22 22]
 [ 1  1 24 24]]


In [6]:
Sigma2_0_inv = 0.0001 * np.eye(p - 1)
beta_0 = np.zeros(p - 1)
nu_0 = 1
sig2_0 = sigma2head_ols

In [7]:
model_string = '''
model {
 # Likelihood
for i in range(1+n) {
y[i] ~ np.random.normal(X[i][0] * beta[0] + X[i][1] * beta[1] + X[i][2] * beta[2] + X[i][3] * beta[3], sigmainv)
}
    
# Priors
sigmainv~ np.random.gamma(nu_0/2, sig2_0*nu_0/2)
beta~ np.random.multivariate_normal(c(0,0,0,0), Sigma2.0.inv)
sigma = inv(sigmainv)
} '''


In [ ]:
model_string = """
model {
  # Likelihood: CAREFUL, in JAGS dnorm(mean, precision)
  for (i in 1:n){
    y[i] ~ dnorm(X[i,1] * beta[1] + X[i,2] * beta[2] + X[i,3] * beta[3] + X[i,4] * beta[4], sigmainv)
  }
  
  # Prior distributions
  sigmainv ~ dgamma(nu_0 / 2, sig2_0 * nu_0 / 2)
  beta ~ dmnorm(beta_0, Sigma2_0_inv)
  sigma = inverse(sigmainv)
}
"""

In [8]:
#Data and initial values to be input in the model

data = {"X": X, "nu_0": nu_0, "sig2_0": sig2_0 , "Sigma2_0_inv": Sigma2_0_inv, "y": y, "n": n}
inits = [{'beta': beta_ols, 'sigma_inv': 1/sigma2head_ols}]

In [19]:
#compile the model, 2 chains to be generated

model = pyjags.Model(model_string, data = dict(data), init = dict(inits), chains = 2)

NameError: name 'pyjags' is not defined

In [ ]:
#Burn-in for 1000 samples for each chain

model.update(1000)

In [ ]:
#Generate 10000 post-burn-in samples for each chain

samples = model.sample(10000)

In [20]:
# Summarize the output

summary = pd.DataFrame(samples['beta'])
summary.columns = ['Intercept', 'beta2', 'beta3', 'beta4']
print(summary.describe())

NameError: name 'update' is not defined

In [9]:
params = ['beta','sigma']

In [10]:
samples = model.sample(10000, vars=['beta','sigma'])

NameError: name 'model' is not defined

In [9]:
pd.DataFrame.as_matrix(samples[[1]][,1:4])
pd.DataFrame.as_matrix(samples[[2]][,1:4])
beta = pd.concat([pd.DataFrame.as_matrix(samples[[1]][,1:4]), pd.DataFrame.as_matrix(samples[[2]][,1:4])])

SyntaxError: invalid syntax (4189598543.py, line 1)

In [11]:
fig, axes = plt.subplots(1, 3)
ax = axes.ravel()
x = np.linspace(start=-85, stop=130, num=200)
plt.plot()

In [ ]:
# Plot the output

plt.figure(figsize=(10, 6))
plt.subplot(1, 3, 1)
plt.hist(samples['beta2'], bins=20)
plt.xlabel('beta2')
plt.axvline(0, color='gray')
plt.subplot(1, 3, 2)
plt.hist(samples['beta4'], bins=20)
plt.xlabel('beta4')
plt.axvline(0, color='gray')
plt.subplot(1, 3, 3)
plt.scatter(samples['beta2'], samples['beta4'])
plt.xlabel('beta2')
plt.ylabel('beta4')
plt.show()

In [ ]:
# Box-plots of beta_2+beta_4*age for different ages

BX = np.empty((0, 12))
for s in range(samples['beta2'].shape[0]):
    BX = np.vstack((BX, (np.arange(20, 32) * samples['beta4'][s] + samples['beta2'][s])))

plt.figure(figsize=(8, 6))
plt.boxplot(BX.T, labels=np.arange(20, 32))
plt.xlabel('age groups from 20 to 31')
plt.ylabel('beta2 + beta4 * age')
plt.axhline(0, color='gray')
plt.show()


In [ ]:
# Diagnostic plots

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(samples['beta2'])
plt.xlabel('Iteration')
plt.ylabel('beta2')
plt.subplot(1, 2, 2)
plt.acorr(samples['beta2'], maxlags=50)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()